In [1]:
# IO
dat_fp = "example/data/"
dat_fn = dat_fp + "primates_cytb.nex"
out_fp = "example/output/"
out_fn = out_fp + "primates"

In [2]:
# MCMC settings
mvi = 1
mni = 1
n_gen = 1e4
sample_freq = floor(n_gen/1e2)

In [3]:
# read alignment
dat = readDiscreteCharacterData(dat_fn)

   Successfully read one character matrix from file 'example/data/primates_cytb.nex'


In [4]:
# data dimensions
taxa = dat.taxa()
n_sites = dat.nchar()
n_taxa = taxa.size()

In [5]:
# create unit Yule tree
birth ~ dnExp(1)
phy ~ dnBDP(lambda=birth, mu=0., rootAge=1, taxa=taxa)

# tree moves
mv[mvi++] = mvNNI(phy, weight=n_taxa)
mv[mvi++] = mvFNPR(phy, weight=n_taxa/2)
mv[mvi++] = mvNodeTimeSlideUniform(phy, weight=n_taxa)

# print the tree's value
phy

  
   ((Lemur_catta[&index=23]:0.659409,Microcebus_murinus[&index=22]:0.659409)[&index=24]:0.340591,(((((((Daubentonia_madagascariensis[&index=21]:0.073606,(Pan_paniscus[&index=20]:0.054335,Saimiri_sciureus[&index=19]:0.054335)[&index=25]:0.019271)[&index=26]:0.314224,Cebus_albifrons[&index=18]:0.387831)[&index=27]:0.078823,Varecia_variegata_variegata[&index=17]:0.466654)[&index=28]:0.122434,(Aotus_trivirgatus[&index=16]:0.154203,(Nycticebus_coucang[&index=15]:0.097088,Loris_tardigradus[&index=14]:0.097088)[&index=29]:0.057115)[&index=30]:0.434885)[&index=31]:0.179212,((Galeopterus_variegatus[&index=13]:0.239766,(Cheirogaleus_major[&index=12]:0.216637,Hylobates_lar[&index=11]:0.216637)[&index=32]:0.023129)[&index=33]:0.343325,((Callicebus_donacophilus[&index=10]:0.226230,Lepilemur_hubbardorum[&index=9]:0.226230)[&index=34]:0.012649,Galago_senegalensis[&index=8]:0.238879)[&index=35]:0.344212)[&index=36]:0.185209)[&index=37]:0.028231,((Perodicticus_potto[&index=7]:0.363053,Propithecus_co

In [6]:
# base frequencies
pi ~ dnDirichlet(rep(1,4))
mv[mvi++] = mvSimplexElementScale(pi, alpha=10, weight=4)

# exchangeability rates
er ~ dnDirichlet(rep(1,6))
mv[mvi++] = mvSimplexElementScale(er, alpha=10, weight=6)

# GTR rate matrix
Q := fnGTR(exchangeRates=er,
           baseFrequencies=pi)
    
# print the matrix's value
Q

   [ [ -0.7399, 0.2148, 0.0204, 0.5047 ] ,
     0.2156, -0.8046, 0.3816, 0.2074 ] ,
     0.0293, 0.5454, -0.9383, 0.3636 ] ,
     1.0808, 0.4425, 0.5428, -2.0661 ] ]


In [7]:
# build phylogenetic CTMC
seq ~ dnPhyloCTMC(tree=phy,
                  Q=Q,
                  branchRates=1.,
                  nSites=n_sites,
                  type="DNA")

# treat the simulated data as the likelihood
seq.clamp(dat)

In [8]:
# create monitors
mn[mni++] = mnScreen(pi, printgen=sample_freq)
mn[mni++] = mnModel(filename=out_fn+".model.log",
                    printgen=sample_freq)
mn[mni++] = mnFile(phy,
                   filename=out_fn+".tre",
                   printgen=sample_freq)

In [9]:
# create MCMC
mdl = model(phy)
ch = mcmc(mdl,mv,mn)

In [10]:
# run MCMC
ch.run(n_gen)


   Running MCMC simulation
   This simulation runs 1 independent replicate.
   The simulator uses 5 different moves in a random move schedule with 67.5 moves per iteration

Iter        |      Posterior   |     Likelihood   |          Prior   |          pi[1]      |       pi[2]      |       pi[3]      |       pi[4]   |    elapsed   |        ETA   |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0           |         -23844   |       -23744.9   |        -99.106   |       0.316428      |    0.315256      |    0.220567      |    0.147749   |   00:00:00   |   --:--:--   |
100         |       -15413.8   |       -15314.9   |       -98.9403   |         0.2417      |    0.277514      |    0.140297      |    0.340489   |   00:00:01   |   --:--:--   |
200         |       -15409.9   |         -15311   |       -98.9352   |       0.247347      |    0.259373      |    0.1